# Realizing topologically ordered states on a quantum processor

## Introduction
Toric code was proposed by Kitaev in paper
"[Fault-tolerant quantum computation by anyons](https://arxiv.org/abs/quant-ph/9707021)"

A good source for study is [Leggett's intro](https://courses.physics.illinois.edu/phys598PTD/fa2013/L26.pdf)
to toric code.

## Ground state preparation

In [ ]:
from toric_code import get_toric_code

x, y = 5, 7  # System size (should be odd)
tc = get_toric_code(x, y)
print(f'Number of qubits: {tc.circ.num_qubits}, depth: {tc.circ.depth()}')
print(f'Plaquette count: {tc.plaquette_x}*{tc.plaquette_y}')

In [1]:
tc.circ.draw()

NameError: name 'tc' is not defined

### Verify ground state preparation
Ground state fulfills $A_s=B_p=1$, i.e., which we can check by applying corresponding operator to plaquettes and stars.


In [ ]:
from toric_code_tests.test_init import get_plaquette_ev

import numpy as np
from qiskit import Aer
from qiskit import transpile

from toric_code import get_toric_code
from toric_code_matching import get_star_matching

In [ ]:
# Use Aer's simulator
backend = Aer.get_backend('aer_simulator')

In [ ]:
px, py = tc.plaquette_x, tc.plaquette_y

for i in range(px):
    for j in range(py):
        ev = get_plaquette_ev(backend, (x, y), (i, j))
        print(f'Plaquette ({i},{j}): {ev}')

In [ ]:
from toric_code_tests.test_init import get_star_ev

sx, sy = tc.star_x, tc.star_y

for i in range(sx):
    for j in range(sy):
        if len(get_star_matching(i, j, tc.y, tc.x)) < 4:
            continue

        ev = get_star_ev(backend, (x, y), (i, j))
        print(f'Star ({i},{j}): {ev}')

### Entropy
To calculate topological entropy ... (explanations on studd)

Here we'll demonstrate calculations for a single $2\times 3$ subsystem.

In [ ]:
from topo_entropy import get_all_2x3_left_non_corner

expected_values = [(2., 2., 2.), (4., 3., 4.), (4.,)]

qubits = get_all_2x3_left_non_corner((x, y))[0]

Pauli basis: measure in every possible combination of Pauli basis ($3^6$ options).

In [ ]:
from topo_entropy import calculate_s_subsystems, ABC_DIVISION_2x3_LEFT
from tqdm import tqdm
import itertools

In [ ]:
all_counts = []

all_gates = [''.join(x) for x in itertools.product('xyz', repeat=len(qubits))]
for gates in tqdm(all_gates):
    tc = get_toric_code(x, y, len(qubits))

    tc.measure_pauli(qubits, gates)

    job = backend.run(transpile(tc.circ, backend), shots=15000)  # note: number of shots is important
    result = job.result()
    counts = result.get_counts(tc.circ)
    all_counts.append(counts)
calculated_values = calculate_s_subsystems(all_counts, ABC_DIVISION_2x3_LEFT)
for expect, calc in zip(expected_values, calculated_values):
    for ve, vc in zip(expect, calc):
        print(f'Theoretical value of Stopo/ln(2): {ve}. Measured value: {vc / np.log(2)} ')

In [ ]:
from tqdm import trange

cnt = 1000
all_counts = []

all_gates = [''.join(x) for x in itertools.product('xyz', repeat=len(qubits))]
for _ in trange(cnt):
    tc = get_toric_code(x, y, len(qubits))

    tc.measure_haar(qubits)

    job = backend.run(transpile(tc.circ, backend), shots=15000)  # note: number of shots is important
    result = job.result()
    counts = result.get_counts(tc.circ)
    all_counts.append(counts)
calculated_values = calculate_s_subsystems(all_counts, ABC_DIVISION_2x3_LEFT)
for expect, calc in zip(expected_values, calculated_values):
    for ve, vc in zip(expect, calc):
        print(f'Theoretical value of Stopo/ln(2): {ve}. Measured value: {vc / np.log(2)} ')

### Braiding
There are three types of quasiparticle $e$, $m$ and $\psi$. We demonstrate braiding of $e$ with $m$,
 as well as fermionic exchange statistics of $\psi$.

In [ ]:
from topo_braiding import apply_cxxxx_on_square, create_e_particles, create_m_particles, apply_cxxyyzz_on_rectangle

In [ ]:
for sq in [(0, 2), (0, 3), (2, 2), (2, 1)]:
    tc = get_toric_code(x, y, classical_bit_count=1, ancillas_count=1)

    x_string = [(2, 1), (2, 2)]
    create_e_particles(tc, x_string)

    z_string = [(1, 4), (0, 3), (1, 3)]  # , (0,2), (1,2)]
    create_m_particles(tc, z_string)

    tc.circ.h(tc.ancillas[0])
    apply_cxxxx_on_square(tc, sq)
    tc.circ.h(tc.ancillas[0])

    tc.circ.measure(tc.ancillas[0], 0)
    Nshots = 10000
    job = backend.run(transpile(tc.circ, backend), shots=Nshots)
    result = job.result()
    counts = result.get_counts(tc.circ)

    if '0' not in counts:
        counts['0'] = 0
    if '1' not in counts:
        counts['1'] = 0

    cos_theta = (counts['0'] - counts['1']) / Nshots
    expected_res = -1. if sq in [(0, 2), (0, 3)] else 1.
    print(f'Expected value of cos(theta): {expected_res}. Measured value: {cos_theta}')

In [ ]:

sq = (0, 3)
tc = get_toric_code(x, y, classical_bit_count=1, ancillas_count=1)

x_strings = [[(2, 1), (2, 2)],
             [(1, 4), (3, 4)]]
for x_string in x_strings:
    create_e_particles(tc, x_string)

z_strings = [[(2, 3), (3, 3), (4, 3)],
             [(3, 1), (4, 1), (3, 2)], ]
for z_string in z_strings:
    create_m_particles(tc, z_string)

tc.circ.h(tc.ancillas[0])
apply_cxxyyzz_on_rectangle(tc, sq)
tc.circ.h(tc.ancillas[0])

tc.circ.measure(tc.ancillas[0], 0)
Nshots = 10000
job = backend.run(transpile(tc.circ, backend), shots=Nshots)
result = job.result()
counts = result.get_counts(tc.circ)

if '0' not in counts:
    counts['0'] = 0
if '1' not in counts:
    counts['1'] = 0
print(counts)

cos_theta = (counts['0'] - counts['1']) / Nshots
expected_res = -1.
print(f'Expected value of cos(theta): {expected_res}. Measured value: {cos_theta}')